In [4]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pickle
import sys
import os

import librosa
import librosa.display

import matplotlib.pyplot as plt

import numpy as np
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import LSTM, Bidirectional, Dense, BatchNormalization, Flatten, merge, MaxPooling1D
from keras.layers import Activation, Concatenate, Permute, Dot, Input, Multiply, RepeatVector, Lambda
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Layer
from keras.layers import MaxPooling1D, AveragePooling1D

from keras.callbacks import EarlyStopping

In [2]:
conda install tensorflow

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\dhars\anaconda3

  added / updated specs:
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _tflow_select-2.3.0        |            eigen           4 KB
    absl-py-0.13.0             |   py38haa95532_0         175 KB
    astor-0.8.1                |   py38haa95532_0          47 KB
    astunparse-1.6.3           |             py_0          17 KB
    blinker-1.4                |   py38haa95532_0          23 KB
    cachetools-4.2.2           |     pyhd3eb1b0_0          13 KB
    coverage-5.5       

tensorflow-base-2.3. | 49.5 MB   | ######2    |  63% 
tensorflow-base-2.3. | 49.5 MB   | ######6    |  66% 
tensorflow-base-2.3. | 49.5 MB   | ######9    |  70% 
tensorflow-base-2.3. | 49.5 MB   | #######2   |  73% 
tensorflow-base-2.3. | 49.5 MB   | #######6   |  76% 
tensorflow-base-2.3. | 49.5 MB   | #######9   |  80% 
tensorflow-base-2.3. | 49.5 MB   | ########3  |  83% 
tensorflow-base-2.3. | 49.5 MB   | ########6  |  87% 
tensorflow-base-2.3. | 49.5 MB   | #########  |  90% 
tensorflow-base-2.3. | 49.5 MB   | #########3 |  93% 
tensorflow-base-2.3. | 49.5 MB   | #########6 |  97% 
tensorflow-base-2.3. | 49.5 MB   | ########## | 100% 

tensorflow-2.3.0     | 6 KB      |            |   0% 
tensorflow-2.3.0     | 6 KB      | ########## | 100% 

rsa-4.7.2            | 28 KB     |            |   0% 
rsa-4.7.2            | 28 KB     | ########## | 100% 

blinker-1.4          | 23 KB     |            |   0% 
blinker-1.4          | 23 KB     | ########## | 100% 
Preparing transaction: ..

In [3]:
conda install keras

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\dhars\anaconda3

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    keras-2.4.3                |       hd3eb1b0_0           6 KB
    keras-base-2.4.3           |     pyhd3eb1b0_0          30 KB
    ------------------------------------------------------------
                                           Total:          36 KB

The following NEW packages will be INSTALLED:

  keras              pkgs/main/noarch::keras-2.4.3-hd3eb1b0_0
  keras-base         pkgs/main/noarch::keras-bas

In [5]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [6]:
walk_dir1="C:/Users/dhars/Documents/The Music Project/genres"

features = np.array([])
genres = np.array([])
for root, subdirs, files in os.walk(walk_dir1):
        for filename in files:
            if filename.endswith('.wav'):
                file_path = os.path.join(root, filename)
                y,sr=librosa.load(file_path)
                zcr=librosa.feature.zero_crossing_rate(y)[:,0:256]
                cent = librosa.feature.spectral_centroid(y=y, sr=sr)[:,0:256]
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)[:,0:256]
                flatness = librosa.feature.spectral_flatness(y=y)[:,0:256]
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)[:,0:256]
                chroma = librosa.feature.chroma_stft(y=y, sr=sr)[:,0:256]
                feat = np.concatenate((mfcc, chroma, spec_bw, flatness, zcr, cent), axis=0)
                feat = feat.T
                feat = feat.reshape((1, feat.shape[0], feat.shape[1]))
                if not features.size:
                    features = np.append(features, feat)
                    features = features.reshape((1, feat.shape[1], feat.shape[2]))
                    genres = np.append(genres, filename.split(".")[0])
                    continue
                features = np.concatenate((features, feat), axis=0)
                genres = np.append(genres, filename.split(".")[0])
                ################
                zcr=librosa.feature.zero_crossing_rate(y)[:,256:512]
                cent = librosa.feature.spectral_centroid(y=y, sr=sr)[:,256:512]
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)[:,256:512]
                flatness = librosa.feature.spectral_flatness(y=y)[:,256:512]
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)[:,256:512]
                chroma = librosa.feature.chroma_stft(y=y, sr=sr)[:,256:512]
                feat = np.concatenate((mfcc, chroma, spec_bw, flatness, zcr, cent), axis=0)
                feat = feat.T
                feat = feat.reshape((1, feat.shape[0], feat.shape[1]))
                features = np.concatenate((features, feat), axis=0)
                genres = np.append(genres, filename.split(".")[0])
                ################
                zcr=librosa.feature.zero_crossing_rate(y)[:,512:768]
                cent = librosa.feature.spectral_centroid(y=y, sr=sr)[:,512:768]
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)[:,512:768]
                flatness = librosa.feature.spectral_flatness(y=y)[:,512:768]
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)[:,512:768]
                chroma = librosa.feature.chroma_stft(y=y, sr=sr)[:,512:768]
                feat = np.concatenate((mfcc, chroma, spec_bw, flatness, zcr, cent), axis=0)
                feat = feat.T
                feat = feat.reshape((1, feat.shape[0], feat.shape[1]))
                features = np.concatenate((features, feat), axis=0)
                genres = np.append(genres, filename.split(".")[0])
                ################
                zcr=librosa.feature.zero_crossing_rate(y)[:,768:1024]
                cent = librosa.feature.spectral_centroid(y=y, sr=sr)[:,768:1024]
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)[:,768:1024]
                flatness = librosa.feature.spectral_flatness(y=y)[:,768:1024]
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)[:,768:1024]
                chroma = librosa.feature.chroma_stft(y=y, sr=sr)[:,768:1024]
                feat = np.concatenate((mfcc, chroma, spec_bw, flatness, zcr, cent), axis=0)
                feat = feat.T
                feat = feat.reshape((1, feat.shape[0], feat.shape[1]))
                features = np.concatenate((features, feat), axis=0)
                genres = np.append(genres, filename.split(".")[0])
                ################
                zcr=librosa.feature.zero_crossing_rate(y)[:,1024:1280]
                cent = librosa.feature.spectral_centroid(y=y, sr=sr)[:,1024:1280]
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)[:,1024:1280]
                flatness = librosa.feature.spectral_flatness(y=y)[:,1024:1280]
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)[:,1024:1280]
                chroma = librosa.feature.chroma_stft(y=y, sr=sr)[:,1024:1280]
                feat = np.concatenate((mfcc, chroma, spec_bw, flatness, zcr, cent), axis=0)
                feat = feat.T
                feat = feat.reshape((1, feat.shape[0], feat.shape[1]))
                features = np.concatenate((features, feat), axis=0)
                genres = np.append(genres, filename.split(".")[0])

In [4]:
walk_dir1="C:/Users/dhars/Documents/The Music Project/genres"
done=0
# dict={}
# data=pd.DataFrame()
features = np.array([])
genres = np.array([])
for root, subdirs, files in os.walk(walk_dir1):
        for filename in files:
            if filename.endswith('.wav'):
                file_path = os.path.join(root, filename)
                y,sr=librosa.load(file_path)
                zcr=librosa.feature.zero_crossing_rate(y)[:,:1270]
                cent = librosa.feature.spectral_centroid(y=y, sr=sr)[:,:1270]
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)[:,:1270]
                flatness = librosa.feature.spectral_flatness(y=y)[:,:1270]
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)[:,:1270]
                chroma = librosa.feature.chroma_stft(y=y, sr=sr)[:,:1270]
                feat = np.concatenate((mfcc, chroma, spec_bw, flatness, zcr, cent), axis=0)
                feat = feat.T
                feat = feat.reshape((1, feat.shape[0], feat.shape[1]))
#                 print(mfcc.shape, feat.shape, root)
                done+=1
                if not features.size:
                    features = np.append(features, feat)
                    features = features.reshape((1, feat.shape[1], feat.shape[2]))
                    genres = np.append(genres, filename.split(".")[0])
#                     print(features.shape)
                    continue
                features = np.concatenate((features, feat), axis=0)
                genres = np.append(genres, filename.split(".")[0])

In [6]:
ohv = np.zeros((10,10))
for i in range(10):
    ohv[i, i] = 1
print(ohv[1])

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
print(filename.split(".")[0])
os.path.join(root, filename)
print(features.shape)
print(genres.shape)

In [ ]:
genres_dict={'rock':ohv[0], 'pop':ohv[1],'classical':ohv[2],'reggae':ohv[3],'disco':ohv[4],'jazz':ohv[5],'metal':ohv[6],'country':ohv[7],'blues':ohv[8],'hiphop':ohv[9]}
genre_id = [genres_dict[item] for item in genres]

In [ ]:
genre_id = np.array(genre_id)
genre_id.flatten()
print(genre_id)

In [10]:
# with open('data.npy', 'wb') as f:
#     np.save(f, features)
#     np.save(f, genre_id)

In [11]:
with open('data.npy', 'rb') as f:
    features = np.load(f)
    genre_id = np.load(f)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, genre_id, test_size=0.2, random_state=42)

In [13]:
(X_train.shape)

(3996, 256, 36)

In [14]:
step_dim = X_train.shape[2]

In [15]:
def AttentionLayer(inputs, step_dim):
    
    attention = Dense(10, activation='tanh')(inputs)
    attention = Dense(1, activation='tanh')(attention)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(step_dim)(attention)
    attention = Permute([2, 1])(attention)
    weighted = Multiply()([attention, inputs])
    weighted_sum = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(256,))(weighted)
    weighted_sum = Dense(10, activation='softmax')(weighted_sum)
#    pooled = MaxPooling1D(2)(weighted)
    
    return weighted_sum

In [16]:
pip install numpy==1.19 --user

Note: you may need to restart the kernel to use updated packages.


In [17]:
max_length = X_train.shape[1]
_input = Input(shape=[max_length, step_dim])

activations = Bidirectional(LSTM(64, return_sequences = True))(_input)
activations = Bidirectional(LSTM(128, return_sequences = True))(_input) 

probabilities = AttentionLayer(activations, 256)


In [18]:
model = Model(inputs = _input, outputs = probabilities)
early_stopping = EarlyStopping(monitor = 'accuracy', verbose = 1, patience = 5)

In [19]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [20]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 36)]    0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 256, 256)     168960      input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 256, 10)      2570        bidirectional_1[0][0]            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 256, 1)       11          dense[0][0]                      
_______________________________________________________________________________________

In [21]:
K.set_value(model.optimizer.learning_rate, 0.0006)

In [22]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 60, validation_data = (X_test, y_test), callbacks = [early_stopping])

Epoch 1/60
125/125 [==============================] - 50s 396ms/step - loss: 2.0196 - accuracy: 0.3001 - val_loss: 1.7680 - val_accuracy: 0.4170
Epoch 2/60
125/125 [==============================] - 41s 332ms/step - loss: 1.7056 - accuracy: 0.4092 - val_loss: 1.6048 - val_accuracy: 0.4650
Epoch 3/60
125/125 [==============================] - 40s 324ms/step - loss: 1.5768 - accuracy: 0.4552 - val_loss: 1.4976 - val_accuracy: 0.4730
Epoch 4/60
125/125 [==============================] - 41s 327ms/step - loss: 1.4904 - accuracy: 0.4847 - val_loss: 1.4131 - val_accuracy: 0.5000
Epoch 5/60
125/125 [==============================] - 43s 346ms/step - loss: 1.4185 - accuracy: 0.5058 - val_loss: 1.3612 - val_accuracy: 0.5370
Epoch 6/60
125/125 [==============================] - 38s 307ms/step - loss: 1.3635 - accuracy: 0.5285 - val_loss: 1.3146 - val_accuracy: 0.5370
Epoch 7/60
125/125 [==============================] - 42s 337ms/step - loss: 1.3335 - accuracy: 0.5325 - val_loss: 1.3017 - val_ac

125/125 [==============================] - 38s 304ms/step - loss: 0.7322 - accuracy: 0.7540 - val_loss: 0.7680 - val_accuracy: 0.7450
Epoch 58/60
125/125 [==============================] - 36s 284ms/step - loss: 0.7096 - accuracy: 0.7625 - val_loss: 0.7848 - val_accuracy: 0.7430
Epoch 59/60
125/125 [==============================] - 34s 269ms/step - loss: 0.7170 - accuracy: 0.7610 - val_loss: 0.7730 - val_accuracy: 0.7330
Epoch 60/60
125/125 [==============================] - 33s 264ms/step - loss: 0.6943 - accuracy: 0.7788 - val_loss: 0.7763 - val_accuracy: 0.7500
